# Download & Setup the data

### Goal of this notebook: Download the large data set into S3 storage

In [1]:
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext

In [2]:
sc

In [3]:
sc.master

u'yarn-client'

In [4]:
#sanity test the cluster
import sys
rdd = sc.parallelize(xrange(10),10)
aa = rdd.map(lambda x: sys.version)
aa.cache()
aa.count()

10

In [14]:
#Setup the S3 bucket for storing the nyc data
!aws s3 mb s3://gdeltdata
!aws s3 mb s3://gdeltdata/data

make_bucket: s3://gdeltdata/
make_bucket: s3://gdeltdata/data/


In [5]:
#Use the ephemerial/tmp space in EC2 node    
!mkdir datafiles
!sudo mount /dev/xvdb datafiles
!mkdir datafiles/data

In [38]:
#20140430.export.CSV.zip

#Setup the variables

baseUrl = "http://data.gdeltproject.org/events/"

#Availaiblity of data set by month & year
yearMonthDict = {}


#availablity for Yellow cab
#yearMonthDict[2015] = range(1,13)
yearMonthDict[2014] = range(1,13)
yearMonthDict[2013] = range(4,13) #daily data available only from April 2014

#file name list
fileUrls = []
filenames = []
for year, monthList in yearMonthDict.iteritems():
    yearStr = str(year)
    for month in monthList:
        monthStr = str(month)
        if len(monthStr) == 1:
            monthStr = "0"+monthStr
        for day in range(1,32):
            dayStr = str(day)
            if len(dayStr) == 1:
                dayStr = "0"+dayStr
            url = baseUrl+yearStr+monthStr+dayStr+".export.CSV.zip"
            fileUrls.append(url)
            #filenames.append(yCabFNPrefix+yearStr+'-'+monthStr+".csv")

In [6]:
#file names from 2013 & backwards
baseUrl = "http://data.gdeltproject.org/events/"

#Availaiblity of data set by month & year
yearMonthDict = {}

yearMonthDict[2013] = range(1,4) #daily data available only from April 2014
yearMonthDict[2012] = range(1,13)
yearMonthDict[2011] = range(1,13)
yearMonthDict[2010] = range(1,13)
yearMonthDict[2009] = range(1,13)
yearMonthDict[2008] = range(1,13)
yearMonthDict[2007] = range(1,13)
yearMonthDict[2006] = range(1,13)

#file name list
fileUrls = []
filenames = []
for year, monthList in yearMonthDict.iteritems():
    yearStr = str(year)
    for month in monthList:
        monthStr = str(month)
        if len(monthStr) == 1:
            monthStr = "0"+monthStr        
        url = baseUrl+yearStr+monthStr+".zip"
        fileUrls.append(url)        

In [12]:
#file names for 2005 and backwards
baseUrl = "http://data.gdeltproject.org/events/"

#Availaiblity of data set by month & year
years = range(1979, 2006)

#file name list
fileUrls = []
for year in years:
    yearStr = str(year)
    url = baseUrl+yearStr+".zip"
    fileUrls.append(url)     

In [ ]:
for url in fileUrls:
    !wget $url -P datafiles/data --trust-server-names

In [ ]:
!unzip datafiles/data/"*.zip" -d datafiles/data/
!rm datafiles/data/*.zip

In [ ]:
#local file system storage utilization
!du -mh datafiles/data

In [ ]:
#add to s3
!aws s3 sync datafiles/data/ s3://gdeltdata/data

In [37]:
#optionally remove files from local system
#!rm datafiles/data/*

In [15]:
#!pip install s3cmd
!s3cmd du s3://gdeltdata/data

113290478675 1115 objects s3://gdeltdata/data


In [16]:
#sanity test
rawEvents = sc.textFile('s3://gdeltdata/data/*').map(lambda x: x.split('\t'))
%time rawEvents.count()
#%time rawEvents.take(2)

CPU times: user 208 ms, sys: 184 ms, total: 392 ms
Wall time: 5min 21s


354518932